In [1]:
%pip install langchain --quiet
%pip install accelerate --quiet
%pip install transformers --quiet
%pip install bitsandbytes --quiet
%pip install unstructured --quiet
%pip install sentence-transformers --quiet
%pip install -U faiss-gpu tiktoken
%pip install langchainhub --quiet
%pip install pinecone-client --quiet
%pip install langchain_pinecone --quiet
%pip install pydantic==1.10 --quiet

In [4]:
import random
import torch
import os
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import pandas as pd
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain import hub
from IPython.display import Markdown, display
import time
import pinecone
from langchain.vectorstores import Pinecone
import pickle
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import GPT4All


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import multi_query_retrieval_helper as mq_helper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [61]:
class MedicalChatbot:
    def __init__(self, embedding_model_name='thenlper/gte-base', llm_model_name="mistralai/Mistral-7B-Instruct-v0.1", faiss_indices_path="../data/faiss_indices", chunks_path="../data/chunked_docs.pkl", k=3, init_all=False, use_pinecone=True):
        self.embedding_model_name = embedding_model_name
        self.llm_model_name = llm_model_name
        self.faiss_indices_path = faiss_indices_path
        self.use_pinecone = use_pinecone
        self.chunks_path = chunks_path
        self.k = k
        self.device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_built() else "cpu"
        if init_all:
            self.init_all()
        self.chat_history = []


    def generate_response(self, user_query):
        response = self.qa_chain({"query": user_query})
        self.chat_history.append(response)
        return response

    def generate_response_with_multi_query(self, user_query):
        response = self.mq_rag_chain({"question": user_query})
        self.chat_history.append(response)
        return response

    def generate_response_with_conversational(self, user_query):
        last_2_query_answer = self.conversational_chat_history[:-3:-1][::-1]
        response = self.conversational_qa_chain({"question": user_query, "chat_history": last_2_query_answer})
        self.chat_history.append(response)
        self.conversational_chat_history.append((user_query, response["answer"]))
        return response




    def init_embedding_model(self):
        model_kwargs = {'device': self.device}
        encode_kwargs = {'normalize_embeddings': True}
        self.embed_model = HuggingFaceEmbeddings(model_name=self.embedding_model_name, model_kwargs=model_kwargs,
                                            encode_kwargs=encode_kwargs)

    def load_bm25_retriever(self):
        with open(self.chunks_path, 'rb') as file:  # Note the 'rb' mode for reading binary files
            chunked_docs = pickle.load(file)
        return BM25Retriever.from_documents(chunked_docs, k=self.k)

    def load_db_retriever(self):
        if self.use_pinecone:
           self.load_Pinecone_db_retriever()
        else:
            self.load_FAISS_db_retriever()

    def load_FAISS_db_retriever(self):
        self.db = FAISS.load_local(f"{self.faiss_indices_path}/{self.embedding_model_name}", self.embed_model)
        self.retriever = self.db.as_retriever(search_type="similarity", search_kwargs={"k": self.k})


    def load_Pinecone_db_retriever(self, index_name="pubmed", search_type="similarity", k=3):
        os.environ["PINECONE_API_KEY"]="1218c885-67e3-492f-b1ab-215405569e97"
        pc = pinecone.Pinecone()
        index = pc.Index(index_name)
        self.db = Pinecone(index, chatbot.embed_model, 'text')
        self.retriever = self.db.as_retriever(search_type=search_type, search_kwargs={"k": k})

    def load_ensemble_retriever(self):
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.load_bm25_retriever(), self.retriever], weights=[0.3, 0.7]
        )


    def get_relevant_documents_for_query(self, query):
        return self.retriever.invoke(query)

    def init_model(self):
        if self.device == "cuda":
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
            self.model=AutoModelForCausalLM.from_pretrained(
            self.llm_model_name, torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map="auto",
            quantization_config=quantization_config
            )

        else:
            self.model = AutoModelForCausalLM.from_pretrained(self.llm_model_name, torch_dtype="auto",  trust_remote_code=True)



    def init_tokenizer(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_name, use_fast=True)
        # self.tokenizer.pad_token = self.tokenizer.eos_token

    def init_pipeline(self):
        generation_config = GenerationConfig.from_pretrained(self.llm_model_name)
        generation_config.max_new_tokens = 512
        generation_config.temperature = 0.001
        generation_config.top_p = 0.95
        generation_config.do_sample = True
        generation_config.repetition_penalty = 1.15

        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            return_full_text=True,
            device=None if self.device=="cuda" else self.device ,
            generation_config=generation_config,
    )

    def init_llm(self):
        self.llm = HuggingFacePipeline(
            pipeline=self.pipeline,
    )


    def init_prompt_template(self):

        # template = """Use the following pieces of context to answer the question at the end.
        # If you don't know the answer, just say that you don't know, don't try to make up an answer.
        # Use three sentences maximum and keep the answer as concise as possible.
        # Always say "thanks for asking!" at the end of the answer.
        #
        # {context}
        #
        # Question: {question}
        #
        # Helpful Answer:"""
        # self.custom_rag_prompt = PromptTemplate.from_template(template)

        self.prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

    def init_qa_chain(self):
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.prompt},
        )

    def init_all(self):
        self.init_embedding_model()
        self.load_db_retriever()
        if "GPT4ALL" in self.llm_model_name.upper():
            self.load_local_gpt4all_model()
        else:
            self.init_tokenizer()
            self.init_model()
            self.init_pipeline()
            self.init_llm()
        self.init_prompt_template()
        self.init_qa_chain()

    def clean_chat_history(self):
        self.chat_history = []

    def init_multi_query_retrieval_chain(self):
        self.mq_output_parser = mq_helper.LineListOutputParser()
        self.mq_prompt_template = mq_helper.get_multi_query_prompt_template()
        self.mq_llm_chain = mq_helper.get_multi_query_llm_chain(chatbot)
        self.mq_retriever = mq_helper.get_multi_query_retriever(chatbot)
        self.mq_retrieval_chain = mq_helper.get_multi_query_retrieval_chain(chatbot)
        self.mq_qa_chain = mq_helper.get_multi_query_qa_chain(chatbot)
        self.mq_rag_chain = mq_helper.get_multi_query_rag_chain(chatbot)
        mq_helper.show_multi_query_logs()

    def init_conversational_retrieval_chain(self):
        self.conversational_chat_history = []

        self.conversational_qa_chain = ConversationalRetrievalChain.from_llm(llm=chatbot.llm,
                                                                verbose=True,
                                                                retriever=chatbot.retriever,
                                                                )
    def load_local_gpt4all_model(self):

        self.llm = GPT4All(
            model=self.llm_model_name,
            max_tokens=2048,
        )





In [ ]:
import time
from IPython.display import Markdown, display

def generate_response_and_measure_time(query, chatbot):
    start_time = time.time()  # Start timing
    result = chatbot.generate_response(query)
    execution_time = time.time() - start_time  # Calculate execution time

    # Print the execution time rounded to 2 decimal places
    print(f"The execution time on {chatbot.device} using {chatbot.llm_model_name} is {round(execution_time, 2)} seconds.\n")

    # Display the query and result
    display(Markdown(f"<b>{result['query']}</b>"))
    display(Markdown(f"<p>{result['result']}</p>"))



## Testing on a local device with MPS:

The accelerator and quantization support only CUDA, so MPS is much slower than CUDA.
- "<b>microsoft/phi-2</b>" works and takes approximately 45 seconds per request.
- "<b>mistralai/Mistral-7B-Instruct-v0.1</b>" does not work due to the error "TypeError: BFloat16 is not supported on MPS."

In [ ]:
chatbot = MedicalChatbot(llm_model_name="microsoft/phi-2")

In [ ]:
chatbot.device

'mps'

In [ ]:
chatbot.init_all()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on mps using microsoft/phi-2 is 41.77 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence. A study conducted in Sweden found that while low premorbid intelligence was associated with lower levels of employment and greater numbers of hospitalizations, it did not explain the higher risk of receiving disability benefits. Additionally, the study found that severe clinical course of bipolar disorder was associated with a higher risk of receiving disability benefits, suggesting that persistent severe symptoms play a larger role in determining occupational outcome than overall cognitive functioning. Therefore, other factors beyond premorbid functioning and intelligence likely contribute to the occupational outcome in individuals with bipolar disorder.
</p>

In [ ]:
chatbot.db

In [ ]:
chatbot.retriever

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x297b97e50>, search_kwargs={'k': 3})

In [ ]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on mps using microsoft/phi-2 is 46.25 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> The effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman are not well established. While some studies have suggested that AAT treatment can improve symptoms of CFS, others have found no significant difference between AAT treatment and placebo. Therefore, it is difficult to make definitive conclusions about the effectiveness of AAT treatment for CFS based on this single case study alone. Further research is needed to determine whether AAT treatment is beneficial for individuals with CFS.
</p>

In [ ]:
chatbot = MedicalChatbot(llm_model_name="mistralai/Mistral-7B-Instruct-v0.1")

In [ ]:
chatbot.device

'mps'

In [ ]:
chatbot.init_all()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: BFloat16 is not supported on MPS

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

In [ ]:
chatbot = MedicalChatbot(llm_model_name="microsoft/phi-2", faiss_indices_path="faiss_indices")

In [ ]:
chatbot.device

'cuda'

In [ ]:
chatbot.init_all()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on cuda using microsoft/phi-2 is 6.11 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence.</p>

In [ ]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on cuda using microsoft/phi-2 is 5.8 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> According to the provided context, there is no direct information about the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS). Therefore, it cannot be determined whether AAT treatment can alleviate the symptoms of CFS based solely on this context. Further research would be needed to determine the effectiveness of AAT treatment on CFS.
</p>

In [ ]:
chatbot = MedicalChatbot(llm_model_name="mistralai/Mistral-7B-Instruct-v0.1",  faiss_indices_path="faiss_indices")

In [ ]:
chatbot.device

'cuda'

In [ ]:
chatbot.init_all()

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 3.11 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence.</p>

In [ ]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 8.96 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> Based on the case study, α1-antitrypsin (AAT) treatment showed improvements in maximal workload, working memory scores, and perceptual organization scores in a 49-year-old woman with chronic fatigue syndrome (CFS). The patient also experienced an improvement in functional capacity, allowing her to work part-time employment. These findings suggest a possible role for AAT in the treatment of CFS.</p>

In [ ]:
query = "Is there a cure to cancer?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 5.72 seconds.


<b>Is there a cure to cancer?</b>

<p> No, there isn't a cure to cancer yet. Current treatment strategies aim at reducing the tumor burden and managing symptoms, but cancer often recurs. Newer therapies like gene editing and AI-powered treatments are being explored, but a definitive cure remains elusive.</p>

In [ ]:
query = "Is there a cure to cancer?"
result = chatbot.generate_response(query)
display(Markdown(f"<b>{result['query']}</b>"))
display(Markdown(f"<p>{result['result']}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Is there a cure to cancer?</b>

<p> No, there isn't a cure to cancer yet. Current treatment strategies aim at reducing the tumor burden and managing symptoms, but cancer often recurs. Newer therapies like gene editing and AI-powered treatments are being explored, but a definitive cure remains elusive.</p>

In [ ]:
display(Markdown(f"<b>Source Documents:</b>"))
display(Markdown(f"<p>{result['source_documents']}</p>"))


<b>Source Documents:</b>

<p>[Document(page_content='Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients cancer will inevitably recur Our critical analysis indicates that it is time to discuss about the new key challenges and future directions in clinical oncology We need to generate novel treatment strategies more suited to the current clinical reality\nAbstract_Length: 1215', metadata={'source': 'processed_data_part1.csv', 'row': 15208, 'PMID': '33312104', 'Title': 'Why we do what we do. A brief analysis of cancer therapies.', 'Authors': 'Carlos M Galmarini', 'Publication Date': '2020', 'DOI': '10.17179/excli2020-2972'}), Document(page_content='Abstract: Cancer is becoming one of the deadliest disease in both developed as well as developing countries and continuous effort is being made to find innovative therapies for myriad types of cancers that afflict the human body Therapeutic options for cancer have grown exponentially over the time but we are quite a way off from finding a magic bullet that can help cure cancer and based on the current evidence we may never find a catch all cure ever and it becomes crucial that we keep on innovating and find multiple ways to attack the menace of this dreaded disease Many patients suffer recurrence of disease and require secondline or in some cases more than two lines of treatment In this review article we have discussed the available therapies along with the newer advancements that have been made in cancer therapy Latest developments in treatment of various cancers that have been discussed include gene editing using CRISPRCas9 theranostics viral mediated therapy artificial intelligence tumor infiltrating lymphocyte therapy etc\nAbstract_Length: 1031', metadata={'source': 'processed_data_part1.csv', 'row': 28024, 'PMID': '34411650', 'Title': 'Futuristic approach to cancer treatment.', 'Authors': 'Deepti Malik; Saniya Mahendiratta; Harpinder Kaur; Bikash Medhi', 'Publication Date': '2021-Dec-30', 'DOI': '10.1016/j.gene.2021.145906'}), Document(page_content='232 7 rhetoric pertaining to the latest technology n63 203 and 8 focusing treatment on cancer origins and cause n43 139 Overall 258 n80 of advertisements included a direct statement claiming provider treatment can cure cancer or prolong life Our results provide evidence alternative cancer providers are using Meta advertising products to market scientifically unsupported cancer treatments Advertisements regularly referenced alternative and natural treatment approaches to cancer Imagery and text content that emulated evidencebased medical providers created the impression that the offered treatments were effective medical options for cancer Advertisements exploited the hope of patients with terminal and poor prognoses by sharing testimonials of past patients who allegedly were cured or had their lives prolonged We recommend that Meta introduce a mandatory humanled authorization process that is not reliant upon artificial intelligence for medicalrelated advertisers before giving advertising permissions Further research should focus on the conflict of interest between social media platforms advertising products and public health', metadata={'source': 'processed_data_part2.csv', 'row': 20798, 'PMID': '37256649', 'Title': 'Advertising Alternative Cancer Treatments and Approaches on Meta Social Media Platforms: Content Analysis.', 'Authors': 'Marco Zenone; Jeremy Snyder; JeanChristophe BelislePipon; Timothy Caulfield; May van Schalkwyk; Nason Maani', 'Publication Date': '2023-May-31', 'DOI': '10.2196/43548'})]</p>

In [ ]:
[r.metadata for r in result['source_documents']]

[{'source': 'processed_data_part1.csv',
  'row': 15208,
  'PMID': '33312104',
  'Title': 'Why we do what we do. A brief analysis of cancer therapies.',
  'Authors': 'Carlos M Galmarini',
  'Publication Date': '2020',
  'DOI': '10.17179/excli2020-2972'},
 {'source': 'processed_data_part1.csv',
  'row': 28024,
  'PMID': '34411650',
  'Title': 'Futuristic approach to cancer treatment.',
  'Authors': 'Deepti Malik; Saniya Mahendiratta; Harpinder Kaur; Bikash Medhi',
  'Publication Date': '2021-Dec-30',
  'DOI': '10.1016/j.gene.2021.145906'},
 {'source': 'processed_data_part2.csv',
  'row': 20798,
  'PMID': '37256649',
  'Title': 'Advertising Alternative Cancer Treatments and Approaches on Meta Social Media Platforms: Content Analysis.',
  'Authors': 'Marco Zenone; Jeremy Snyder; JeanChristophe BelislePipon; Timothy Caulfield; May van Schalkwyk; Nason Maani',
  'Publication Date': '2023-May-31',
  'DOI': '10.2196/43548'}]

# Testing Chat History

In [ ]:
chatbot = MedicalChatbot(llm_model_name="microsoft/phi-2", init_all=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
chatbot.generate_response(query)
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
chatbot.generate_response(query)
query = "Is there a cure to cancer?"
chatbot.generate_response(query)
print("")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for response in chatbot.chat_history:
    display(Markdown(f"<b>{response['query']}</b>"))
    display(Markdown(f"<p>{response['result']}</p>"))
    print("****************************************************")

<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence. While studies have shown that individuals with bipolar disorder often experience difficulties in their careers due to the nature of the condition, it appears that other factors play a larger role in determining employment status. These include the severity of the individual's symptoms, the type of job they hold, and whether or not they receive appropriate treatment. Additionally, socioeconomic factors such as income and education level can also influence one’s chances of finding stable employment. Therefore, while premorbid functioning and intelligence may contribute to overall success in life, these two variables alone do not appear to be sufficient indicators of future career prospects for someone living with bipolar disorder.</p>

****************************************************


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> The effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman are not well established. While some studies have suggested that AAT treatment can improve symptoms of CFS, others have found no significant difference between AAT treatment and placebo. Therefore, it is difficult to make definitive conclusions about the effectiveness of AAT treatment for CFS based on this single case study alone. Further research is needed to determine whether AAT treatment is beneficial for individuals with CFS.
</p>

****************************************************


<b>Is there a cure to cancer?</b>

<p> The advertisement claims that the provided treatment can cure cancer or prolong life. It uses imagery and text content that emulates evidence-based medical providers to create the impression that the offered treatments are effective medical options for cancer. Testimonials of past patients who allegedly were cured or had their lives prolonged are shared to exploit the hope of patients with terminal and poor prognoses. Meta recommends introducing a mandatory human-led authorization process for medical-related advertisers before granting advertising permissions. Further research should focus on the conflict of interest between social media platforms advertising products and public health.
</p>

****************************************************


In [ ]:
chatbot.clean_chat_history()

In [ ]:
for response in chatbot.chat_history:
    display(Markdown(f"<b>{response['query']}</b>"))
    display(Markdown(f"<p>{response['result']}</p>"))
    print("****************************************************")

In [27]:
chatbot = MedicalChatbot(llm_model_name="/Users/yusuf/Library/Application Support/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf")

In [28]:
chatbot.init_all()

In [18]:
chatbot.init_multi_query_retrieval_chain()

In [21]:
query = "Is there a cure for cancer?"
response = chatbot.generate_response_with_multi_query(query)
response["text"]

INFO:langchain.retrievers.multi_query:Generated queries: ['-----------------------------------------------', '    Alternative 1: What is the current state of cancer research?', '    Alternative 2: How can we prevent cancer from spreading?', '    Alternative 3: Can we use AI to predict and treat cancer?']


'\n---\nThere is no single cure for cancer as it is a complex disease with various subtypes and stages However there are many treatments available that can help manage and reduce symptoms and improve quality of life for patients with cancer These include surgery radiation therapy chemotherapy immunotherapy and targeted therapies among others The choice of treatment depends on the type and stage of cancer as well as individual patient factors such as age overall health and personal preferences\n'

In [62]:
chatbot = MedicalChatbot(llm_model_name="/Users/yusuf/Library/Application Support/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf")

In [63]:
chatbot.init_all()

In [64]:
chatbot.init_conversational_retrieval_chain()

In [65]:
query = "How does regular physical exercise influence cognitive function and intelligence in adults?"
response = chatbot.generate_response_with_conversational(query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The present study examined the effects of current physical exercise and age on episodic memory and fluid intelligence assessed with a freerecall task and the Culture Fair Intelligence Test Cattell 1963 respectively while statistically controlling for other cognitive reserve factors educational level leisure activities and vocabulary level Two hundred and eight participants aged 20 to 85 participated in the study Physical exercise level was indexed by weekly frequency over the last 12 months using selfreported measurement from none to 4 times a week Overall results show a beneficial effect of physical exercise especially from a weekly practice of 2 times and significant interaction between physical exercise and age on epi

In [66]:
response["answer"]

' Regular physical exercise has been shown to improve cognitive function and intelligence in adults. A metaanalysis of 10 studies found that aerobic or resistance training interventions significantly improved cognitive ability in older adults compared with control interventions. The study also found that physical exercise improved cognitive abilities at different physiological levels, including intelligence quotient IQ and factors related to physical fitness PF in healthy adults with ID. However, more research is needed to enhance the understanding of the relationships between PF and cognition in persons with ID.'

In [67]:
query = "Thank you so much, this is a great answer!"
response = chatbot.generate_response_with_conversational(query)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How does regular physical exercise influence cognitive function and intelligence in adults?
Assistant:  Regular physical exercise has been shown to improve cognitive function and intelligence in adults. A metaanalysis of 10 studies found that aerobic or resistance training interventions significantly improved cognitive ability in older adults compared with control interventions. The study also found that physical exercise improved cognitive abilities at different physiological levels, including intelligence quotient IQ and factors related to physical fitness PF in healthy adults with ID. However, more research is needed to enhance the understanding of the relationships between PF and cognition in persons with ID.
Follow Up Input: Thank you so much, this is a gr

In [70]:
response["answer"]

'  Regular physical exercise has been shown to improve cognitive function and intelligence in adults with intellectual disabilities. A study conducted by researchers at multiple centers found that individuals with intellectual disabilities who engaged in a cognitive training program showed significant improvement in their fluid ability, which is an important aspect of cognitive functioning. The study also found that the physical activity level of older adults with intellectual disabilities is extremely low, and their fitness levels are far beneath accepted norms for older people with normal intelligence and comparable with frail older people. A physical activity program including an education program was developed for older adults with ID using behavior change techniques. The program aimed to improve or maintain adequate levels of physical activity, primary outcome measure, and motor fitness, cardio respiratory fitness, morphologic and metabolic fitness activities of daily living, cogn